# RAG

首先加载本地模型

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain import LLMChain,HuggingFacePipeline,PromptTemplate
import torch
model_path = "/home/workspace_langchain/medical-chatbot-main/medical-chatbot"
# load model from huggingface，load因果语言类大模型的时候就用这个函数 AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(
        model_path,
        load_in_4bit=True,
        torch_dtype=torch.float16,
        device_map='auto'
)

tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

from transformers import pipeline

# 构建 transformer 的 pipeline
pipe = pipeline(
    "text-generation",  # 名称
    model=model,  # 模型
    torch_dtype=torch.bfloat16,  # 模型加载方式
    device_map='auto',
    max_new_tokens=512,  # 最大token
    do_sample=True,
    top_k=30,
    num_return_sequences=1,
    tokenizer=tokenizer,
    eos_token_id=tokenizer.eos_token_id
)

llm = HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature': 0})

The repository for /home/workspace_langchain/medical-chatbot-main/medical-chatbot contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co//home/workspace_langchain/medical-chatbot-main/medical-chatbot.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y
The repository for /home/workspace_langchain/medical-chatbot-main/medical-chatbot contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co//home/workspace_langchain/medical-chatbot-main/medical-chatbot.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

The model 'ChatGLMForConditionalGeneration' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'Musicge

In [2]:
from langchain_community.document_loaders import TextLoader # 文本加载器
from langchain.text_splitter import CharacterTextSplitter # 文本分块器
from langchain_community.embeddings import OllamaEmbeddings # Ollama向量嵌入器
import weaviate # 向量数据库
from weaviate.embedded import EmbeddedOptions # 向量嵌入选项
from langchain.prompts import ChatPromptTemplate # 聊天提示模板
from langchain_community.chat_models import ChatOllama # ChatOllma聊天模型
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser # 输出解析器
from langchain_community.vectorstores import Weaviate # 向量数据库
import requests

# 加载不同的数据

In [3]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import PyPDFLoader
# 读取csv，返回list
def load_csv(path):
    # 每条记录为一个元素
    loader = CSVLoader(
        file_path=path,
        encoding='utf-8' # 编码
    )
    data = loader.load()
    return data 
# 读取pdf，返回list
def load_pdf(path):
    # 是以每页为一个元素的
    loader = PyPDFLoader(path)
    pages = loader.load_and_split()
    return pages

# 数据切片


In [4]:
loader = TextLoader("/home/workspace_langchain/rag/内科.txt", encoding='utf-8')
documents = loader.load()

In [ ]:
# 切分txt
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=10,separator="，")

chunks1 = text_splitter.split_documents(documents)

In [10]:
len(chunks1)

9492

In [4]:
# 切分不同的文本
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100, # 指定每个文本块的目标大小，这里设置为200个字符。
    chunk_overlap=10, # 指定文本块之间的重叠字符数，这里设置为50个字符。
    length_function=len, # 用于测量文本长度的函数，这里使用Python内置的`len`函数。
    is_separator_regex=False, # 指定`separators`中的分隔符是否应被视为正则表达式，这里设置为False，表示分隔符是字面字符。
    separators=["\n\n",  "\n",   " ",    ".",    ",",     "，",  "。", ] # 定义用于分割文本的分隔符列表。csv不用这行
)

pages = load_pdf("/home/workspace_langchain/Gong/data/深度学习.pdf")

chunks2 = text_splitter.split_documents(pages)
len(chunks2)

10581

In [7]:
# 合并 chunks1 和 chunks2
chunks = []
chunks.extend(chunks1)
chunks.extend(chunks2)
len(chunks)

20073

# 将数据存入向量数据库中

In [ ]:
# 嵌入和存储这些小块
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.embedded import EmbeddedOptions

client = weaviate.Client(
  embedded_options = EmbeddedOptions()
)

vectorstore = Weaviate.from_documents(
    client = client,    
    documents = chunks2,
    # 本地加载
    embedding = HuggingFaceEmbeddings(model_name='/home/workspace_langchain/text2vec-base-chinese'),#注意加载的问题
    by_text = False
)


# 进行增强检索

In [53]:
# 步骤1——检索
retriever = vectorstore.as_retriever()

In [3]:
# 步骤2——增强
from langchain.prompts import ChatPromptTemplate

template = """你是一个用于回答问题的助手。使用以下检索到的Context来回答问题。如果你不知道答案，就说你不知道。使用最多三个句子并保持回答简洁。
如果是学术知识，给出答案所在page和相关内容。如果是关于医疗知识，给出科室地址和相关诊断。如果Context和问题无关，则忽略Context内容。

Question: {question}


Context: {context}

Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

print(prompt)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='你是一个用于回答问题的助手。使用以下检索到的Context来回答问题。如果你不知道答案，就说你不知道。使用最多三个句子并保持回答简洁。\n如果是学术知识，给出答案所在page和相关内容。如果是关于医疗知识，给出科室地址和相关诊断。如果Context和问题无关，则忽略Context内容。\n\nQuestion: {question}\n\n\nContext: {context}\n\nAnswer:\n'))]


# 进行搜索

In [ ]:
rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser() 
)

query = "神经网络的好处"

rag_chain.invoke(query)